# Example profiling of `empymod`

In [1]:
import numpy as np
from IPython.core import page

import empymod as epm

# Disable the pager so output is stored in notebook
page.page = print

# Load magics
%load_ext memory_profiler
%load_ext line_profiler

### Define Model

In [2]:
model = {
    'src': [0, 0, 400],
    'rec': [np.arange(1,1001)*10, np.arange(1,1001)*3, 400],
    'depth': [ 0, 200, 500, 600, 1100, 1300, 1800, 2300, 3000],
    'res': [2e14, 4, 10, 20, 1/.2, 1/.3, 1/.4, 1/.5, 1/.6, 1/.7],
    'aniso': [.5, 4, 2, 1.5, 3, .25, 2, .5, 1, 2],
    'freqtime': np.arange(1,21),
    'epermH': [1, 80, 5, 10, 20, 3, 20, 5, 8, 10],
    'epermV': [1, 80, 5, 20, 10, 8, 10, 5, 6, 5],
    'mpermH': [1, 1, 1.0001, 1, 1, 1.00001, 1, 1, 1, 1],
    'mpermV': [1, 1, 1, 1.0001, 1.0001, 1, 1, 1, 1, 1],
    'ab': 11, 
    'ht': 'FHT',
    'htarg': [epm.filters.key_401_2009(), -1],
    'verb': 1,
    'xdirect': False,
}

## Profile

Activate or adjust depending on what you want to profile

In [3]:
%lprun -f epm.greenfct epm.dipole(**model)
#%lprun -f epm.greenfct -f epm.reflections -f epm.fields epm.dipole(**model)

Timer unit: 1e-06 s

Total time: 0.083664 s
File: /home/dtr/Codes/empymod/empymod/kernel.py
Function: greenfct at line 136

Line #      Hits         Time  Per Hit   % Time  Line Contents
   136                                           def greenfct(zsrc, zrec, lsrc, lrec, depth, etaH, etaV, zetaH, zetaV, lambd,
   137                                                        ab, xdirect, msrc, mrec, use_ne_eval):
   138                                               """Calculate Green's function for TM and TE.
   139                                           
   140                                               .. math:: \\tilde{g}^{tm}_{hh}, \\tilde{g}^{tm}_{hz},
   141                                                         \\tilde{g}^{tm}_{zh}, \\tilde{g}^{tm}_{zz},
   142                                                         \\tilde{g}^{te}_{hh}, \\tilde{g}^{te}_{zz}
   143                                           
   144                                               This function c

## Memory and other tests

In [4]:
print('timeit')
%timeit epm.dipole(**model)

print('\nmemit')
%memit epm.dipole(**model)

print('\nmprun')
%mprun -f epm.greenfct epm.dipole(**model)

print('\nprun')
%prun -l 10 epm.dipole(**model)

timeit
120 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

memit
peak memory: 96.40 MiB, increment: 2.69 MiB

mprun
Filename: /home/dtr/Codes/empymod/empymod/kernel.py

Line #    Mem usage    Increment   Line Contents
   136     93.9 MiB     93.9 MiB   def greenfct(zsrc, zrec, lsrc, lrec, depth, etaH, etaV, zetaH, zetaV, lambd,
   137                                          ab, xdirect, msrc, mrec, use_ne_eval):
   138                                 """Calculate Green's function for TM and TE.
   139                             
   140                                 .. math:: \\tilde{g}^{tm}_{hh}, \\tilde{g}^{tm}_{hz},
   141                                           \\tilde{g}^{tm}_{zh}, \\tilde{g}^{tm}_{zz},
   142                                           \\tilde{g}^{te}_{hh}, \\tilde{g}^{te}_{zz}
   143                             
   144                                 This function corresponds to equations 108--110, 117/118, 122; 89--94,
   145                  

In [5]:
epm.versions('HTML')